### Import dependencies

In [23]:
import pandas as pd
import json
import numpy as np
import re
from flatten_json import flatten
import pandas_profiling as pp

### Parse the csv file as a pandas dataframe object

In [2]:
df = pd.read_csv('mongo.csv')
df.head()

,Unnamed: 0,model_data,analyzed,filename,status,createdAt,updatedAt,videoUploadType,gpu_id,analysed,text,home_team,away_team
0,0,"{'team1_possession': 17, 'team2_possession': 0...",True,30milsec.mp4,NaN,2021-09-16 08:07:21.604,2021-09-16 09:40:05.040,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"{'TeamA': {'Players': []}, 'TeamB': {'Players'...",False,30milsec.mp4,NaN,2021-09-16 11:43:04.129,2021-09-16 11:43:04.129,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"{'team1_possession': 0, 'team2_possession': 0,...",True,30milsec.mp4,NaN,2021-09-16 11:46:26.110,2021-09-16 22:34:17.335,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"{'TeamA': {'Players': []}, 'TeamB': {'Players'...",False,30milsec.mp4,NaN,2021-09-16 11:48:59.938,2021-09-16 11:48:59.938,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"{'TeamA': {'Players': []}, 'TeamB': {'Players'...",False,30milsec.mp4,NaN,2021-09-16 11:50:13.634,2021-09-16 11:50:13.634,NaN,NaN,NaN,NaN,NaN,NaN


### Drop all rows with NaN in '_model_data_' column

In [3]:
print(len(df))
df = df.dropna(subset=['model_data'])
len(df)

255


254

### Define a function to turn the string values in model_column into double quotes as per proper json format 
#### (_NB: Having keys with single quotes is not proper format for a JSON object_)

In [4]:
def replace_single_quotes(string):
    s = re.sub( "(?<={)\'|\'(?=})|(?<=\[)\'|\'(?=\])|\'(?=:)|(?<=: )\'|\'(?=,)|(?<=, )\'", "\"", string)
    return s

#### _Apply function to rows of "model_data" column_

In [5]:
df['model_data'] = df['model_data'].apply(lambda x: replace_single_quotes(x))

### Turn booleans and objects within nested dicts into strings

In [6]:
df['model_data'] = df['model_data'].apply(lambda x: re.sub(' Ob[^,]+', " \" Obj\"", x))
df['model_data'] = df['model_data'].apply(lambda x: re.sub(' True', "\" True\"", x))

### Jsonify "model_data" column   i.e   cast each row of model_data as a dictionary

In [7]:
df['model_data'] = df['model_data'].apply(json.loads)

In [8]:
df['model_data'][0:5]

0    {'team1_possession': 17, 'team2_possession': 0...
1    {'TeamA': {'Players': []}, 'TeamB': {'Players'...
2    {'team1_possession': 0, 'team2_possession': 0,...
3    {'TeamA': {'Players': []}, 'TeamB': {'Players'...
4    {'TeamA': {'Players': []}, 'TeamB': {'Players'...
Name: model_data, dtype: object

### Flatten the dictionary values

In [9]:
df['model_data'] = df['model_data'].apply(lambda x: flatten(x))

#### _Make a backup copy of the dataframe for data checks_

In [10]:
df1 = df.copy()
df1

,Unnamed: 0,model_data,analyzed,filename,status,createdAt,updatedAt,videoUploadType,gpu_id,analysed,text,home_team,away_team
0,0,"{'team1_possession': 17, 'team2_possession': 0...",True,30milsec.mp4,NaN,2021-09-16 08:07:21.604,2021-09-16 09:40:05.040,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"{'TeamA_Players': [], 'TeamB_Players': [], 'ur...",False,30milsec.mp4,NaN,2021-09-16 11:43:04.129,2021-09-16 11:43:04.129,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"{'team1_possession': 0, 'team2_possession': 0,...",True,30milsec.mp4,NaN,2021-09-16 11:46:26.110,2021-09-16 22:34:17.335,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"{'TeamA_Players': [], 'TeamB_Players': [], 'ur...",False,30milsec.mp4,NaN,2021-09-16 11:48:59.938,2021-09-16 11:48:59.938,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"{'TeamA_Players': [], 'TeamB_Players': [], 'ur...",False,30milsec.mp4,NaN,2021-09-16 11:50:13.634,2021-09-16 11:50:13.634,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,"{'TeamA_Players': [], 'TeamB_Players': [], 'ur...",False,file upload,NaN,2022-04-26 14:08:27.133,2022-04-26 14:08:27.133,NaN,NaN,NaN,NaN,NaN,NaN
251,251,"{'TeamA_Players': [], 'TeamB_Players': [], 'ur...",False,file upload,NaN,2022-04-26 17:01:59.707,2022-04-26 17:01:59.707,NaN,NaN,NaN,NaN,NaN,NaN
252,252,"{'TeamA_Players': [], 'TeamB_Players': [], 'ur...",False,URL upload,NaN,2022-04-26 17:07:15.428,2022-04-26 17:07:15.428,NaN,NaN,NaN,NaN,NaN,NaN
253,253,"{'TeamA_Players': [], 'TeamB_Players': [], 'ur...",False,URL upload,NaN,2022-04-26 17:09:42.114,2022-04-26 17:09:42.114,NaN,NaN,NaN,NaN,NaN,NaN


#### _Check 1_

In [11]:
df1['model_data'][1]

{'TeamA_Players': [],
 'TeamB_Players': [],
 'url_object_detection': '',
 'url_minimap': '',
 'url_team': '',
 'url_weight': '',
 'url_height': '',
 'url_color': '',
 'url_formation': '',
 'url_position': '',
 'url_name': '',
 'url_jerseynumber': '',
 'isFootballVideo': ' True',
 'team1_possession': 0,
 'team2_possession': 0,
 'team1_penalty': 0,
 'team2_penalty': 0,
 'team1_longpass': 0,
 'team2_longpass': 0,
 'team1_cross': 0,
 'team2_cross': 0,
 'team1_freethrow': 0,
 'team2_freethrow': 0,
 'team1_shortpass': 0,
 'team2_shortpass': 0,
 'team1_foul': 0,
 'team2_foul': 0,
 'team1_cornerkick': 0,
 'team2_cornerkick': 0,
 'team1_free_kick': 0,
 'team2_free_kick': 0,
 'team1_longshot': 0,
 'team2_longshot': 0,
 'team1_yellowcard': 0,
 'team2_yellowcard': 0,
 'team1_shots': 0,
 'team2_shots': 0,
 'team1_save': 0,
 'team2_save': 0,
 'team1_dribble': 0,
 'team2_dribble': 0,
 'team1_redcard': 0,
 'team2_redcard': 0,
 'team1_tackle': 0,
 'team2_tackle': 0,
 'team1_goals': 0,
 'team2_goals': 0

### Apply _json_normalize_ method on the "model_data" column 

In [12]:
df2 = pd.json_normalize(df1['model_data'])
df2.head()

,team1_possession,team2_possession,team1_penalty,team2_penalty,team1_longpass,team2_longpass,team1_cross,team2_cross,team1_freethrow,team2_freethrow,...,TeamB_Players_10_Distance_13,TeamB_Players_10_Distance_19,TeamB_Players_10_Distance_22,TeamB_Players_10_Distance_25,TeamB_Players_10_Distance_28,TeamB_Players_10_Distance_31,TeamB_Players_10_Distance_34,TeamB_Players_10_Distance_37,TeamB_Players_10_Distance_40,TeamB_Players_10_Distance_43
0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### _Check 2_

In [13]:
df2.iloc[0:2, 30:40]

,team1_goals,team2_goals,TeamA_Players_0_Team,TeamA_Players_0_Position,TeamA_Players_0_Name,TeamA_Players_0_Jersey_no,TeamA_Players_0_Weight,TeamA_Players_0_Height,TeamA_Players_0_Color,TeamA_Players_0_Formation
0,1.0,0.0,par,gk,Alphonse Areola,1,88kg,192cm,darkgrey,0.0
1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df2.describe()

,team1_possession,team2_possession,team1_penalty,team2_penalty,team1_longpass,team2_longpass,team1_cross,team2_cross,team1_freethrow,team2_freethrow,...,TeamB_Players_10_Distance_13,TeamB_Players_10_Distance_19,TeamB_Players_10_Distance_22,TeamB_Players_10_Distance_25,TeamB_Players_10_Distance_28,TeamB_Players_10_Distance_31,TeamB_Players_10_Distance_34,TeamB_Players_10_Distance_37,TeamB_Players_10_Distance_40,TeamB_Players_10_Distance_43
count,254.000000,254.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
mean,15.361067,6.184901,1.652000,0.444000,1.924000,0.628000,0.704000,0.312000,10.372000,2.640000,...,0.034831,0.140248,0.023116,0.124852,0.003219,0.027025,0.105264,0.090619,0.086847,0.028558
std,29.750785,15.036553,10.606265,3.480131,10.752333,3.709536,3.826612,3.039013,76.138492,25.230313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.034831,0.140248,0.023116,0.124852,0.003219,0.027025,0.105264,0.090619,0.086847,0.028558
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.034831,0.140248,0.023116,0.124852,0.003219,0.027025,0.105264,0.090619,0.086847,0.028558
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.034831,0.140248,0.023116,0.124852,0.003219,0.027025,0.105264,0.090619,0.086847,0.028558
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.034831,0.140248,0.023116,0.124852,0.003219,0.027025,0.105264,0.090619,0.086847,0.028558
max,99.968750,87.500000,108.000000,49.000000,130.000000,52.000000,49.000000,47.000000,855.000000,320.000000,...,0.034831,0.140248,0.023116,0.124852,0.003219,0.027025,0.105264,0.090619,0.086847,0.028558


In [17]:
df2.columns[(df2 == 0).all()]

Index([], dtype='object')

In [20]:
df2.isnull().sum()

team1_possession                  0
team2_possession                  0
team1_penalty                     4
team2_penalty                     4
team1_longpass                    4
                               ... 
TeamB_Players_10_Distance_31    253
TeamB_Players_10_Distance_34    253
TeamB_Players_10_Distance_37    253
TeamB_Players_10_Distance_40    253
TeamB_Players_10_Distance_43    253
Length: 56720, dtype: int64

In [26]:
profile40 = pp.ProfileReport(df2.iloc[:,0:40])
profile40.to_file("output40.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Personal observations

- The number of resulting columns from the model_data column is quite large (over 56,000). <br>
Did some checks to trace if this was a result of errors in normalisation but couldn't find any.<br>
<br>
- My suspicion though is this is due to the highly irregular nested structure of the json column.<br>
Perhaps some rows have an even deeper nested structure <br>
causing normalisation with a dataframe bearing the maximum number of column titles <br>
to lead to the dataframe having a lot of columns with NaNs and zero values.<br>


- A row within the original model_data column is a null value<br>
<br>
- Some of the keys within the JSON had values which were booleans (True) or raw object descriptions (Object.....)